In [1]:
import sys
sys.path.append('..')
from Comms_System import Comms_System
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import scipy.stats
from tqdm import tqdm
from ML_components import load_params
from Network import NeuralNetwork

## Automatic Tests

In [2]:
symbol_set = [7, 5, 3, 1, -1, -3, -5, -7] # all symbols that we use
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)

decisions = CS.transmission(noise_level=1, mode='euclidean')
CS.evaluate(decisions)

(57, 0.0057)

### Manual Test

In [3]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=0.35)

# calibrate
gain_factor = np.max(np.convolve(CS.h, CS.h))

# upsample symbol sequence and filter it on transmission side
upsampled = CS.upsample(v=False)
Tx = np.convolve(upsampled, CS.h)

# Transmit the filtered signal (i.e. add noise)
Tx = Tx + np.random.normal(0.0, 0, Tx.shape)  # add gaussian noise

# Filter on receiver side
Rx = np.convolve(Tx, CS.h)

# Downsample the signal on the receiver side
downsampled = CS.downsample(Rx)

# Decision-making using new_values
decisions = CS.decision_making(downsampled/gain_factor, False)

CS.evaluate(decisions)

(0, 0.0)

## Display filter

In [ ]:
betas = [0, 0.35, 0.7, 1]
plt.figure(figsize=(13,8))
plt.title('Root Raised Cosine', fontsize=24)
plt.xlabel('Time', fontsize=20)
plt.ylabel('Amplitude', fontsize=20)
z = np.zeros(len(CS.h))
for beta in betas:
    CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=beta)
    #f = np.fft.fft(CS.h)
    plt.plot(CS.h)
plt.grid(True)
plt.legend(betas, fontsize=12)
plt.show()


plt.figure(figsize=(13,8))
plt.title('Raised Cosine', fontsize=24)
plt.xlabel('Time', fontsize=20)
plt.ylabel('Amplitude', fontsize=20)
z = np.zeros(len(CS.h))
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=0.35)
plt.stem(np.convolve(CS.h, CS.h))
plt.grid(True)
plt.show()

____

# Evaluation of pretrained Decision Making model

In [4]:
for i in range(10):

    symbol_set = [3, 1, -1, -3] # all symbols that we use
    num_symbols = 10000
    symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
    m = 8
    CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
    noise_level = 1

    # Automatic test
    decisions, NN_decisions, downsampled, block_decisions = CS.test_CS(noise_level=noise_level, v=False)
    print('Euclidean Distance Error Rate: {}%'.format((CS.evaluate(decisions)[1]*100).round(2)))
    #print('Neural Net Error Rate: {}%'.format((CS.evaluate(NN_decisions)[1]*100).round(2)))
    print('Neural Net Block Error Rate: {}%'.format((CS.evaluate(block_decisions)[1]*100).round(2)))
    print()

/Users/Dragonborn/Documents/Master Thesis/Code/DE_PyCharm/Jupyter


FileNotFoundError: [Errno 2] No such file or directory: '../SavedWeights/block_decision_making_weights.npz'

### SNR Calculations

In [ ]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=0.35)

sigmas = np.linspace(0.75, 4.5, 10) #sigmas = np.linspace(2.5, 4.5, 500)#
SNRs = []
error_rates = []
error_rates_NN = []
error_rates_NN_blocks = []
avg_symbol_energy = np.mean(np.array(symbol_seq)**2)
gain_factor = np.max(np.convolve(CS.h, CS.h))

for sigma in sigmas:

    decisions, NN_decisions, downsampled, NN_block_decisions = CS.test_CS(noise_level=sigma, v=False)
    SNRs.append(avg_symbol_energy*gain_factor/(sigma**2))
    error_rates.append(CS.evaluate(decisions)[1])
    error_rates_NN.append(CS.evaluate(NN_decisions)[1])
    error_rates_NN_blocks.append(CS.evaluate(NN_block_decisions)[1])
    
    
SNRsDB = 10*np.log10(SNRs)
error_rates = np.array(error_rates)
error_rates_NN = np.array(error_rates_NN)
error_rates_NN_blocks = np.array(error_rates_NN_blocks)
error_theory = 1.5 * (1 - scipy.stats.norm.cdf(np.sqrt(gain_factor/sigmas**2))) # 

In [ ]:
plt.figure(figsize=(13,8))
plt.title('Decision Making Noise Plots', fontsize=24)
plt.xlabel('SNR (dB)', fontsize=20)
plt.ylabel('$P_e$', fontsize=20)
num = 0
plt.semilogy(SNRsDB[num:], error_rates[num:], alpha=1)
#plt.semilogy(SNRsDB[num:], error_rates_NN[num:], alpha=1)
plt.semilogy(SNRsDB[num:], error_rates_NN_blocks[num:])
plt.semilogy(SNRsDB[num:], error_theory[num:])
legend = ['Euclidean', 'Neural Network', 'Neural Network Blocks' ,'Theory']
plt.legend(legend, fontsize=12)
#plt.ylim([1e-3, 1])
plt.show()

# Inspect Decision Boundaries

In [ ]:
weights, biases, sizes = load_params('../SavedWeights/decision_making_weights.npz',
                                         '../SavedWeights/decision_making_biases.npz')
NN = NeuralNetwork(sizes, startweights=weights, startbiases=biases,
                           type='classification', afunc='relu')
classes = np.array(symbol_set)

## Plot 1

In [ ]:
test_seq = np.linspace(-4, 4, 1000)
test_seq = np.array(test_seq, ndmin=2).T
yhat_onehot = NN.feedforward(test_seq)


plt.figure(figsize=(13,8))
plt.title('Decision Boundaries', fontsize=24)
for i in range(yhat_onehot.shape[1]):
    plt.plot(test_seq, yhat_onehot[:,i], '-x')
#plt.axvline(x=-2, color='black')
#plt.axvline(x=0, color='black')
#plt.axvline(x=2, color='black')
plt.xlabel('Input', fontsize=20)
plt.ylabel('Probability of choosing', fontsize=20)
plt.legend(symbol_set + ['Euclidean Boundaries'], fontsize=14)
plt.show()

## Plot 2

In [ ]:
classes = np.array(symbol_set)
test_seq = np.linspace(-4, 4+1, 100)
test_seq = np.array(test_seq, ndmin=2).T

yhat_idx = NN.feedforward(test_seq).argmax(axis=1)
yhat = classes[yhat_idx]
euclid_decisions = CS.decision_making(test_seq)


num = len(test_seq)
plt.figure(figsize=(13,8))
plt.xlabel('Sample Value (input)', fontsize=20)
plt.ylabel('Decided value (output)', fontsize=20)
plt.plot(test_seq[:num], yhat[:num], '-rx', alpha=0.5)
plt.plot(test_seq[:num], euclid_decisions[:num], '-bx', alpha=1)
plt.legend(['Neural Network', 'Euclidean'], fontsize=14)
plt.show()

##### Want to time something? Use following code stump

In [ ]:
# %%timeit -o -r 5